In [1]:
import sympy as sp
import numpy as np
import galois as gf
import pandoc
from math import *

np.set_printoptions(legacy='1.25')

## Problem 1

Consider the matrix

$ A = \begin{bmatrix}
3 & 1 & 6 \\
3 & 0 & 4 \\
1 & 3 & 2 
\end{bmatrix}  $

in GF(7).

(a) Find the determinant of $A$. Is it invertible?

>A matrix is invertible if it is a square $n$ by $n$ matrix, and the determinant is non-zero.

In [37]:
# det| a b | = a*e - b*d
#    | d e |

#matrix: numpy array.
#p: galois field value GF(p)
def determinant_2x2_GF(matrix,p) :
    return np.mod(int(np.linalg.det(matrix)),p)

#matrix: numpy array.
#p: galois field value GF(p)
def determinant_3x3_GF(matrix,p) :
    return np.mod(int(np.linalg.det(matrix)),p)

In [39]:
# A = np.array([[0,4],[3,2]])
# print(f'Testing determinant_2x2_GF...')
# print(f'Expected value = 2. Answer = {determinant_2x2_GF(A,7)}')

A = np.array([[3,1,6],[3,0,4],[1,3,2]])

print(f'The determinant of A = {determinant_3x3_GF(A,7)}')

The determinant of A = 2


In [45]:
print(len(A))

3


> The determinant of A = 2, which is not zero, therefore A is invertible.

> I used numpy's numpy.linalg.det() function which returns the determinant of a matrix. I take the modulus of the result.

(b) Recall that in real numbers, the inverse of $A$ can be found as

$ A^{-1} = \frac{1}{det(A)} \begin{bmatrix}
det(A(1, 1)) & − det(A(1, 2)) & det(A(1, 3)) \\
− det(A(2, 1)) & det(A(2, 2)) & − det(A(2, 3)) \\
det(A(3, 1)) & − det(A(3, 2)) & det(A(3, 3)) 
\end{bmatrix}  $

where $A(i, j)$ is the matrix obtained by removing row $i$ and column $j$ of matrix $A$. For instance,

$ A(1,2) = \begin{bmatrix}
3 & 4 \\
1 & 2 
\end{bmatrix}  $

Note that $±$ det($A(i, j)$) is still an integer, and hence, you can find the inverse of $A$ in GF(7) by replacing division by det($A$) by multiplying by in the multiplicative inverse of det($A$) in GF(7). Now, use the method above to find the inverse of $A$ in GF(7). You may use MATLAB or any other software. Describe your method.

In [144]:
#Finds inverse in modulo. e.g. 2 inverse in mod 3 is 2 b/c 2*2 = 4(mod 3) = 1
#==Inputs== 
#input: value to find the inverse of
#modulo: value of the modulus
def find_modulo_inverse(input, modulo) :
    i=modulo
    while i>0 :
        if np.mod(input*i,modulo) == 1 :
            return i
        i-=1
    return 0

def modulo_divide(a, b, m):
    a = np.mod(a,m)
    inv = find_modulo_inverse(b,m)
    if inv == 0 :
        print("Division not Defined")
        return 0
    else :
        return np.mod(inv*a,m)


#matrix: nxn numpy array.
#p: p in GF(p)
#returns the inversem matrix.
def inverse_matrix_GF(matrix, p) :
    determinant_inv = find_modulo_inverse(int(np.mod(np.linalg.det(matrix),p)),p)
    output = matrix*0
    i=0
    negative = 1 # +/- 1 depending on the row/column index.
    while i<len(output):
        j=0
        while j<len(output) :
            print(f'det_inv = {determinant_inv}')
            print(f'output[i,j] = {determinant_inv*negative*int(np.linalg.det(np.delete((np.delete(matrix, i,axis=0)),j,axis=1)))}')
            output[i,j] = np.mod((determinant_inv*negative*int(np.linalg.det(np.delete((np.delete(matrix, i,axis=0)),j,axis=1)))),p) #deletes row i and column j, and finds the determinant of the resulting matrix, and divides the determinant into the result.
            negative = negative*-1 #alternates the negative sign to the matrix
            j+=1
        i+=1
    return output

In [154]:
A = np.array([[3,1,6],[3,0,4],[1,3,2]])
A_inv = inverse_matrix_GF(A,7)

np.matmul(A,A_inv)


det_inv = 4
output[i,j] = -48
det_inv = 4
output[i,j] = -8
det_inv = 4
output[i,j] = 36
det_inv = 4
output[i,j] = 64
det_inv = 4
output[i,j] = 0
det_inv = 4
output[i,j] = -32
det_inv = 4
output[i,j] = 16
det_inv = 4
output[i,j] = 24
det_inv = 4
output[i,j] = -12


array([[16, 36, 18],
       [11, 30, 11],
       [ 8, 12, 14]])

det

$ A^{-1} = \begin{bmatrix}
1 & 6 & 1 \\
1 & 0 & 3 \\
2 & 3 & 2 
\end{bmatrix}  $


## Problem 2

Consider a general block cipher algorithm that maps $n$ bits to $n$ bits.

(a) How many different such mapping (including the identity map) exist?

(b) Let $K(n)$ be your answer to part (a). Bob and Alice want to communicate privately. In order to set the mapping, they shared a long table including all possible $K(n)$ mappings and associated a number between $0$ and $K(n) − 1$ to each. Then, they wrote the corresponding number for each mapping in Binary representation. For instance, if $K(n) = 1000$, mapping number $347$ is represented by $’0101011011’$. Then, each time Bob wants to send a private message to Alice, he chooses one of these mappings and sends her its binary representation through a secure channel. How many bits does Bob have to send for each mapping?

(c) Recall that we mentioned in class that $n$-to-$n$ bits mappings can be also represented by $n$ × $2n$ bits (similar to Table. 4.1 of the textbook). Is this number less or greater than what you found in part (b)? Explain the discrepancy.

## Problem 3

Consider the GF(28) generated by $q(x) = x^{8} + x^{7} + x^{2} + x + 1$. Evaluate the following quantities:

(a) $\{BC\} + \{2E\}$.

(b) $\{0A\} \cdot \{A3\}$.

(c) $\{84\}−1$

## Problem 4

Consider GF($3^{3}$) generated by $m(x) = x3 + 2x2 + 1$, and let $\alpha$ be a root of $m(x)$. Let $a = 2\alpha^{2} + \alpha + 1$. We have already seen that we can find the multiplicative inverse of $a$ as follows: Let $b = b_{2}\alpha^{2} + b_{1}\alpha + b_{0}$ is $a^{−1}$. Then, we there should exists some $q$ such that

$a \cdot b = m(α) \cdot q + 1.$

(a) We can argue that $q$ should be of the form of $q = q_{1}\alpha + q_{0}$. Why $q$ does not have any term of $\alpha^{2}$?

Now, to solve for (1), we can write

$a \cdot b = (2\alpha^{2} + \alpha + 1) \cdot (b_{2}\alpha^{2} + b_{1}\alpha + b_{0}) = (2b_{2})\alpha^{4} + (2b_{1} + b_{2})\alpha^{3} + (2b_{0} + b_{1} + b_{0})\alpha^{2} + (b_{1} + b_{0})\alpha + b_{0}$

$m(α) \cdot q + 1 = q_{1}\alpha^{4} + (2q_{1} + q_{0})\alpha^{3} + (2q_{0})\alpha^{2} + q_{1}\alpha + (q_{0}) + 1.$

So, we need to determine $(b_{2}, b_{1}, b_{0})$ and $(q_{1}, q_{0})$ such that

$2b_{2} = q_{1}$

$2b_{1} + b_{2} = 2q_{1} + q_{0}$

$2b_{0} + b_{1} + b_{0} = 2q_{0}$

$b_{1} + b_{0} = q_{1}$

$b_{0} = q_{0} + 1.$

(b) Write the equations in (2) as a linear system, i.e, $A \cdot x = b$, where $x = [b_{0}, b_{1}, b_{2}, q_{0}, q_{1}]^{T}$ . Find $A$ and $b$.

(c) Use Problem 1 to find the inverse of $A$ and solve the system for $x$. What is $b$?

(d) Write a computer program that computes the multiplicative inverse in a general finite field. Consider GF($p^{k}$) generated by $m(x) = m_{k}x^{k} + · · · + m_{1}x + m_{0}$ which is represented as $M = [m_{k}, m_{k−1}, . . . , m_{1}, m_{0}]$. For a value $a = a_{k−1}\alpha^{k−1} + · · · + a_{1}\alpha + a_{0}$, which is represented as $A = [a_{k−1}, . . . , a_{1}, a_{0}]$, compute $b = b_{k−1}\alpha^{k−1} + · · · + b_{1}\alpha + b_{0}$ such that $a \cdot b = 1$ and return $B = [b_{k−1}, . . . , b_{1}, b_{0}]$.


## Problem 5

Recall the S-box used in the Advanced Encryption Standard (AES). It maps a pair $\{xy\}$ to $\{x^{′}y^{′}\}$ = $S(\{xy\})$ according to the following steps:

• You first find {uv} = {xy}−1.

• Write {uv} in binary {uv} = [b7, . . . , b0].

• Compute

$ \begin{bmatrix}
b_{0}^{'} \\
b_{1}^{'} \\
b_{2}^{'} \\
b_{3}^{'} \\
b_{4}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
b_{0}^{'} \\
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
1 & 1 & 0 & 0 & 0 & 1 & 1 & 1 \\
1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 
\end{bmatrix} \cdot
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2} \\
b_{3} \\
b_{4} \\
b_{5} \\
b_{6} \\
b_{7} 
\end{bmatrix} +
\begin{bmatrix}
1 \\
1 \\
0 \\
0 \\
0 \\
1 \\
1 \\
0 
\end{bmatrix}$ (mod 2)

• Compute $\{x^{′}y^{′}\} = [b^{′}_{7}, . . . , b^{′}_{0}]$.

Write a computer program to generate the S-box. It should take $\{xy\}$ and return $\{x^{′}y^{′}\}$.